평당월세 계산 ("KIC_price.csv"파일 있으면 flow 시작가능)

In [1]:
import pandas as pd

df = pd.read_csv('KIC_price.csv')

평당월세 = (df.iloc[:, 3] / df.iloc[:, 6]).round(2) 

df.insert(5, '평당월세', 평당월세)

df.to_csv('KIC_price_1.csv', index=False, encoding='utf-8-sig')

print("평당월세 데이터를 계산하여 KIC_price_1.csv로 저장했습니다.")

평당월세 데이터를 계산하여 KIC_price_1.csv로 저장했습니다.


In [3]:
import pandas as pd

df = pd.read_csv('KIC_price_1.csv')

print(f"행개수 : {len(df)}")

행개수 : 19032


지번주소 찾기

In [6]:
import pandas as pd
import requests
import time

# 데이터 읽기
df = pd.read_csv('KIC_price_1.csv')

# Kakao REST API Key
REST_API_KEY = 'f32e9311c06e941799af2d38345741de'
location_cache = {}

def reverse_geocode(lon, lat):
    """위도/경도를 기반으로 지번 주소를 조회하는 함수"""
    location_key = f"{lon}_{lat}"
    
    # 캐시 확인
    if location_key in location_cache:
        return location_cache[location_key]
    
    # API 호출
    url = 'https://dapi.kakao.com/v2/local/geo/coord2address.json'
    headers = {'Authorization': f'KakaoAK {REST_API_KEY}'}
    params = {'x': lon, 'y': lat}
    
    try:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            result = response.json()
            if result['documents']:
                address = result['documents'][0].get('address')
                address_result = address['address_name'] if address else '주소 정보 없음'
                location_cache[location_key] = address_result
                return address_result
            else:
                location_cache[location_key] = '주소 정보 없음'
                return '주소 정보 없음'
        else:
            return f'오류 발생: {response.status_code}'
    except Exception as e:
        return f'오류 발생: {str(e)}'
    finally:
        time.sleep(0.5)

# 위도/경도 열이 있는지 확인
if '위도' in df.columns and '경도' in df.columns:
    df['지번주소'] = ''  # 새로운 "지번주소" 열 추가
    
    for idx, row in df.iterrows():
        lon = row['경도']  # 경도
        lat = row['위도']  # 위도
        
        if pd.notna(lon) and pd.notna(lat):
            address = reverse_geocode(lon, lat)
            df.loc[idx, '지번주소'] = address
        else:
            df.loc[idx, '지번주소'] = '좌표 정보 없음'
        
        if idx % 10 == 0:
            print(f"처리 중: {idx}/{len(df)}")
    
    # 결과 저장
    df.to_csv('KIC_price_2.csv', index=False, encoding='utf-8-sig')
    print("지번 주소 정보를 추가하여 KIC_price_지번주소.csv로 저장했습니다.")
    
    # 캐시 및 API 호출 정보 출력
    unique_locations = len(location_cache)
    total_locations = len(df)
    print(f"API 호출 횟수: {unique_locations}")
else:
    print("데이터프레임에 '위도'와 '경도' 열이 없습니다.")

처리 중: 0/19032
처리 중: 10/19032
처리 중: 20/19032
처리 중: 30/19032
처리 중: 40/19032
처리 중: 50/19032
처리 중: 60/19032
처리 중: 70/19032
처리 중: 80/19032
처리 중: 90/19032
처리 중: 100/19032
처리 중: 110/19032
처리 중: 120/19032
처리 중: 130/19032
처리 중: 140/19032
처리 중: 150/19032
처리 중: 160/19032
처리 중: 170/19032
처리 중: 180/19032
처리 중: 190/19032
처리 중: 200/19032
처리 중: 210/19032
처리 중: 220/19032
처리 중: 230/19032
처리 중: 240/19032
처리 중: 250/19032
처리 중: 260/19032
처리 중: 270/19032
처리 중: 280/19032
처리 중: 290/19032
처리 중: 300/19032
처리 중: 310/19032
처리 중: 320/19032
처리 중: 330/19032
처리 중: 340/19032
처리 중: 350/19032
처리 중: 360/19032
처리 중: 370/19032
처리 중: 380/19032
처리 중: 390/19032
처리 중: 400/19032
처리 중: 410/19032
처리 중: 420/19032
처리 중: 430/19032
처리 중: 440/19032
처리 중: 450/19032
처리 중: 460/19032
처리 중: 470/19032
처리 중: 480/19032
처리 중: 490/19032
처리 중: 500/19032
처리 중: 510/19032
처리 중: 520/19032
처리 중: 530/19032
처리 중: 540/19032
처리 중: 550/19032
처리 중: 560/19032
처리 중: 570/19032
처리 중: 580/19032
처리 중: 590/19032
처리 중: 600/19032
처리 중: 610/19032
처리 중: 620/19032
처리 

"지번주소"열과 "기타정보"열의 위치를 바꾸기


In [8]:
import pandas as pd

# 데이터 읽기
df = pd.read_csv('KIC_price_2.csv')

# 열 이름 확인
if '지번주소' in df.columns and '기타정보' in df.columns:
    # 열 위치 변경
    columns = list(df.columns)
    address_index = columns.index('지번주소')
    other_info_index = columns.index('기타정보')
    
    # "주소"와 "기타정보" 열 위치를 교환
    columns[address_index], columns[other_info_index] = columns[other_info_index], columns[address_index]
    df = df[columns]
    
    # 결과 저장
    df.to_csv('KIC_price_3.csv', index=False, encoding='utf-8-sig')
    print("주소 열과 기타정보 열의 위치를 바꾸어 KIC_price_3.csv로 저장했습니다.")
else:
    print("'주소' 또는 '기타정보' 열이 데이터프레임에 없습니다.")

주소 열과 기타정보 열의 위치를 바꾸어 KIC_price_3.csv로 저장했습니다.


지우고싶은열 지우기

In [10]:
import pandas as pd

# 데이터 읽기
df = pd.read_csv('KIC_price_3.csv')

columns_to_drop = ['기타정보', '층수'] 

df = df.drop(columns=columns_to_drop)

# 결과 저장
df.to_csv('KIC_price_4.csv', index=False, encoding='utf-8-sig')
print(f"{columns_to_drop} 열을 삭제하고 KIC_price_4.csv로 저장했습니다.")

['기타정보', '층수'] 열을 삭제하고 KIC_price_4.csv로 저장했습니다.


In [11]:
import pandas as pd
import re

# 데이터 로드
data = pd.read_csv('KIC_price_4.csv')

# '지번주소' 열에서 숫자가 나오는 순간부터 맨 뒤까지 제거
data['지번주소'] = data['지번주소'].apply(lambda x: re.sub(r'\d.*', '', x).strip())

# 결과 확인
print(data['지번주소'].head())

# 수정된 데이터 저장
data.to_csv('KIC_price_4_modified.csv', index=False, encoding='utf-8-sig')
print("수정된 데이터를 KIC_price_4_modified.csv로 저장했습니다.")

0    강원특별자치도 원주시 반곡동
1    강원특별자치도 원주시 반곡동
2    강원특별자치도 원주시 반곡동
3    강원특별자치도 원주시 반곡동
4    강원특별자치도 원주시 반곡동
Name: 지번주소, dtype: object
수정된 데이터를 KIC_price_4_modified.csv로 저장했습니다.


--------

In [25]:
import pandas as pd

# 파일 로드
cluster_data = pd.read_csv('동으로 묶기 가능.csv', encoding='utf-8')

# 1열(index=0)의 값 수정: "경기"를 "경기도"로 변경
cluster_data.iloc[:, 0] = cluster_data.iloc[:, 0].str.replace('울산', '울산광역시', regex=False)

# 결과 저장
cluster_data.to_csv('동으로 묶기 가능.csv', index=False, encoding='utf-8-sig')
print("1열의 '경기'를 '경기도'로 수정하여 동으로 묶기 가능_수정.csv로 저장했습니다.")

1열의 '경기'를 '경기도'로 수정하여 동으로 묶기 가능_수정.csv로 저장했습니다.


In [ ]:
import pandas as pd

가능_df = pd.read_csv("동으로 묶기 가능.csv")
KIC_df = pd.read_csv("KIC.csv")

KIC_df["클러스터"] = ""

for index, row in 가능_df.iterrows():
    동 = row["동"]
    
    KIC_df.loc[KIC_df["주소"].str.contains(동, na=False), "클러스터"] = 동

KIC_df.to_csv("결과.csv", index=False, encoding='utf-8-sig')

print("작업 완료! 결과는 '결과.csv' 파일에 저장되었습니다.")

작업 완료! 결과는 '결과.csv' 파일에 저장되었습니다.


In [29]:
import pandas as pd

# 데이터 읽기
KIC_df = pd.read_csv("KIC.csv", encoding="utf-8-sig")

# 열 이름 확인
print("현재 열 순서:", KIC_df.columns.tolist())

# "클러스터" 열을 원하는 위치로 이동 (예: 3열과 4열 사이로 이동)
columns = KIC_df.columns.tolist()
cluster_index = columns.index("클러스터")  # "클러스터" 열의 현재 위치
columns.remove("클러스터")  # 기존 위치에서 제거
columns.insert(3, "클러스터")  # 원하는 위치에 삽입 (파이썬 index 기준)

# 열 순서 변경
KIC_df = KIC_df[columns]

# 결과 저장
KIC_df.to_csv("결과_클러스터_위치변경.csv", index=False, encoding="utf-8-sig")

print("클러스터 열의 위치를 변경하여 결과_클러스터_위치변경.csv로 저장했습니다.")

현재 열 순서: ['번호', '센터명', '주소', '위도', '경도', '은행', '편의점', '지하철 개수', '지하철 점수', '지하철등급', '국도(km)', '고속도로(km)', '도로점수', '도로등급', '클러스터']
클러스터 열의 위치를 변경하여 결과_클러스터_위치변경.csv로 저장했습니다.


---------------

매물데이터에 지산 넣기

In [4]:
import pandas as pd
import numpy as np


est = pd.read_csv("KIC_price.csv")
kic = pd.read_csv("KIC_cluster.csv")
from IPython.display import display

display(est.head())
display(kic.head())

,지역,물건번호,상가구분,월세,보증금,평당월세,계약면적,전용면적,지번주소,위도,...,region,C_min,C_max,전세금(J),P_min,P_max,예상 매매가,단위 면적당 매매가,층,전체층
0,강원특별자치도 원주시 반곡동,2528873558,지식산업센터,40,500.0,1.11,56,36.18,강원특별자치도 원주시 반곡동 1809-11,37.327746,...,비수도권,5.5,6.5,9227.272727,7807.692308,9227.272727,8517.482517,235.419638,8,15
1,강원특별자치도 원주시 반곡동,2528886552,지식산업센터,40,500.0,1.04,76,38.49,강원특별자치도 원주시 반곡동 1809-11,37.327746,...,비수도권,5.5,6.5,9227.272727,7807.692308,9227.272727,8517.482517,221.290790,7,15
2,강원특별자치도 원주시 반곡동,2528480164,지식산업센터,40,500.0,0.90,88,44.59,강원특별자치도 원주시 반곡동 1809-11,37.327746,...,비수도권,5.5,6.5,9227.272727,7807.692308,9227.272727,8517.482517,191.017773,4,15
3,강원특별자치도 원주시 반곡동,2528476385,지식산업센터,40,500.0,0.94,42,42.69,강원특별자치도 원주시 반곡동 1809-11,37.327746,...,비수도권,5.5,6.5,9227.272727,7807.692308,9227.272727,8517.482517,199.519384,3,15
4,강원특별자치도 원주시 반곡동,2528332848,지식산업센터,90,1000.0,1.18,152,76.54,강원특별자치도 원주시 반곡동 1809-11,37.327746,...,비수도권,5.5,6.5,20636.363636,17461.538462,20636.363636,19048.951049,248.875765,10,15


,번호,센터명,주소,클러스터,위도,경도,은행,편의점,지하철 개수,지하철 점수,지하철등급,국도(km),고속도로(km),도로점수,도로등급
0,0,강릉 공공임대형 지식산업센터,강원특별자치도 강릉시 사천면 방동리 805-3,11,37.806207,128.854086,E,E,0,0.0,E,0.91,7.81,1.63,A
1,1,동해지식산업센터,강원특별자치도 동해시 구호동 216,30,37.481576,129.133811,C,E,0,0.0,E,3.65,6.42,4.65,E
2,2,표준공장1동,강원특별자치도 동해시 구호동 247번지,30,37.480842,129.153095,E,E,0,0.0,E,5.65,9.70,7.14,E
3,3,표준공장2동,강원특별자치도 동해시 구호동 247번지,30,37.480842,129.153095,E,E,0,0.0,E,5.65,9.70,7.14,E
4,4,표준공장3동,강원특별자치도 동해시 구호동 247번지,30,37.480842,129.153095,E,E,0,0.0,E,5.65,9.70,7.14,E


In [5]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from tqdm import tqdm

# 데이터 로드
est = pd.read_csv("KIC_price.csv")      # 매물 데이터
kic = pd.read_csv("KIC_cluster.csv")    # 지산 데이터

# 결과 열 추가
est['속한지산이름'] = -1
est['클러스터'] = -1

# 주소 포함 매칭 (1차)
for idx, est_row in tqdm(est.iterrows(), total=len(est)):
    matched = False
    est_addr = str(est_row['지번주소'])
    for kic_idx, kic_row in kic.iterrows():
        kic_addr = str(kic_row['주소'])
        if kic_addr in est_addr or est_addr in kic_addr:
            est.at[idx, '속한지산이름'] = kic_row['센터명']
            est.at[idx, '클러스터'] = kic_row['클러스터']
            matched = True
            break
    if matched:
        continue

    # 2차: 위경도 거리 기반 최근접(200m 이내)
    if pd.notna(est_row['위도']) and pd.notna(est_row['경도']):
        est_coord = (est_row['위도'], est_row['경도'])
        min_dist = float('inf')
        min_kic_idx = -1
        for kic_idx, kic_row in kic.iterrows():
            if pd.notna(kic_row['위도']) and pd.notna(kic_row['경도']):
                kic_coord = (kic_row['위도'], kic_row['경도'])
                dist = geodesic(est_coord, kic_coord).meters
                if dist < min_dist:
                    min_dist = dist
                    min_kic_idx = kic_idx
        if min_dist <= 200:
            est.at[idx, '속한지산이름'] = kic.at[min_kic_idx, '센터명']
            est.at[idx, '클러스터'] = kic.at[min_kic_idx, '클러스터']
        # else: 이미 -1로 되어 있음

# 결과 저장
est.to_csv("KIC_price_with_cluster.csv", index=False, encoding='utf-8-sig')
print("매물데이터에 속한지산이름, 클러스터 열을 추가하여 KIC_price_with_cluster.csv로 저장했습니다.")

  0%|          | 0/19032 [00:00<?, ?it/s]/var/folders/gh/rmj80qln5tz2v55hd102xqhr0000gn/T/ipykernel_13739/1907740386.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '엔터비즈타워' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  est.at[idx, '속한지산이름'] = kic_row['센터명']
100%|██████████| 19032/19032 [59:47<00:00,  5.31it/s] 


매물데이터에 속한지산이름, 클러스터 열을 추가하여 KIC_price_with_cluster.csv로 저장했습니다.


In [7]:
import pandas as pd

df = pd.read_csv("KIC_price_with_cluster.csv")

print(df['클러스터'].value_counts())

클러스터
28     1194
41     1039
127    1005
17      848
123     793
       ... 
80        2
14        1
69        1
8         1
167       1
Name: count, Length: 109, dtype: int64
